In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

In [ ]:
data = pd.read_csv('images.csv')

In [ ]:
data = data[data['kids'] == False]
data = data[['image', 'label']]

In [ ]:
image_dir = 'images_original/'

In [ ]:
img_size = (100, 100)

In [ ]:
from PIL import Image
import os
import numpy as np

def load_images(file_paths, img_size):
    images = []
    failed_images = []
    for file_path in file_paths:
        file_path_without_extension = os.path.splitext(file_path)[0]  # Remove the file extension
        full_path = os.path.join(image_dir, f"{file_path_without_extension}.jpg")
        try:
            with Image.open(full_path) as img:
                img = img.convert('RGB')
                img = img.resize(img_size)
                img_array = np.array(img)
                images.append(img_array)
        except Exception as e:
            print(f"Error loading image: {file_path}. Exception: {e}")
            failed_images.append(file_path)
    return np.array(images), failed_images

# Update this variable with the correct directory
image_dir = 'images_compressed'  # Adjust if needed
image_paths = data['image'].values  # Assuming this contains all image file names

# Assuming img_size is defined somewhere (e.g., img_size = (224, 224))

loaded_images, failed_list = load_images(image_paths, img_size)

print(f"Successfully loaded {len(loaded_images)} images.")
print(f"Failed to load {len(failed_list)} images.")


In [ ]:
print(f"Successfully loaded {len(loaded_images)} images.")
print(f"Failed to load {len(failed_list)} images.")

In [ ]:
image_paths = data['image'].values
images = load_images(image_paths, img_size)

In [1]:
labels = pd.get_dummies(data['label'])

NameError: name 'pd' is not defined

In [ ]:
from sklearn.model_selection import train_test_split

# Assuming labels are in a pandas Series called 'labels'
# Convert labels to a numpy array if they're not already in that format
labels_array = labels.to_numpy()  # or labels.values, depending on your setup

# Split the data
train_images, test_images, train_labels, test_labels = train_test_split(images, labels_array, test_size=0.2, random_state=42)

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(len(labels.columns), activation='softmax')  # Output layer with the number of classes
])

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [2]:
history = model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_data=(test_images, test_labels))

NameError: name 'model' is not defined

In [ ]:
def extract_dominant_color(img):
    pixels = img.reshape((-1, 3))
    pixels = np.float32(pixels)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 200, 0.1)
    k = 3  # Number of clusters (can be adjusted)
    _, labels, centers = cv2.kmeans(pixels, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    dominant_color = centers[0]  # Extract the dominant color
    return dominant_color